## RAG Pipeline using LlamaIndex
This notebook will create a RAG pipeline using LlamaIndex. There are 5 key stages in RAGs listed as: Loading the data, Indexing the data, Storing the data, Querying the data, and then Evaluation

#### Loading and Embedding Documents
There are three main ways to load data into LlamaIndex:

1. SimpleDirectoryReader: A built-in loader for various file types from a local directory.
2. LlamaParse: LlamaParse, LlamaIndex’s official tool for PDF parsing, available as a managed API.
3. LlamaHub: A registry of hundreds of data-loading libraries to ingest data from any source.


In [1]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir = "Documents")     # Add proper path to where documents are stored
documents = reader.load_data()

After loading the documents, we will break them into smaller pieces called Nodes. We will use the IngestionPipeline to help us create these nodes. We will use SentenceSplitter to break down documents into manageable chunks and HuggingFaceEmbedding to convert each chunk into numerical embeddings.

In [6]:
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations = [
        SentenceSplitter(chunk_overlap = 0),
        HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5"), #Make sure you are login to HuggingFace and have a token, you can see it from the terminal by using huggingface-cli login
    ]
)

# The pipeline defined here is SentenceSplitter -> HuggingFaceEmbeddings

nodes = await pipeline.arun(documents = documents)
# This will save the transformed chunks of documents into the node variable

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\DELL\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\AppData\Local\llama_index\llama_index\Cache\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]